In [1]:
from IPython.core.display import display, HTML, Javascript


display(HTML('<h1><div id="text">Hello, world!</div></h1>'))
Javascript("document.getElementById('text').innerHTML = 'voila'")

<IPython.core.display.Javascript object>

In [2]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [3]:
from ipykernel.comm import Comm

my_comm = Comm(target_name='klampt_comm', data={'foo': 1})
my_comm.send({'foo': 2})

# Add a callback for received messages.
@my_comm.on_msg
def _recv(msg):
    # Use msg['content']['data'] for the data in the message
    print "Received in Python",msg['content']['data']

In [9]:
from IPython.core.display import display, HTML, Javascript

container = """<div id="canvas" style="width:800px; height:600px; margin:0; padding:0">container</div>"""

code2 = """
    function loadScript(url,cb,failcb) {
        jQuery.ajax({
            url: url,
            dataType: 'script',
            success: cb,
            failure: failcb,
            async: true
        });
    }
    function loadScripts(urls,finalcb) {
        if(urls.length==1) {
            loadScript(urls[0],finalcb,function() { alert("Failed to load "+urls[0]); });
        }
        else {
            loadScript(urls[0],function() { 
                    loadScripts(urls.slice(1,urls.length),finalcb);
                },
                function() {
                    alert("Failed to load "+urls[0]);
                })
        }
    }
    var prefix = "http://klampt.org/";
    var myklampt = null;
    loadScripts([prefix+"three.min.js",
                prefix+"KlamptFrontend.js"],
             function() {
                myklampt = new KLAMPT(document.getElementById("canvas")); 

                Jupyter.notebook.kernel.comm_manager.register_target('klampt_comm',
                function(comm, msg) {
                    // comm is the frontend comm instance
                    // msg is the comm_open message, which can carry data
                    comm.on_msg(function(msg) {
                        var payload = msg['content']['data']
                        if(payload.type == 'transforms') {
                            myklampt.update_scene(payload.data);
                        }
                        else if(payload.type == 'scene') {
                            myklampt.set_scene(payload.data);
                        }
                        else if(payload.type == 'rpc') {
                            myklampt.rpc(payload.data);
                        }
                        else {
                            console.log("Illegal message type "+payload.type);
                        }
                    });
                });
    });
"""

display(HTML(container))
Javascript(code2)


<IPython.core.display.Javascript object>

In [10]:

msg = {'type':'rpc',
       'data':{'type':'add_text','name':'test','text':'hello','x':10,'y':10}}
my_comm = Comm(target_name='klampt_comm')
my_comm.send(msg)

In [11]:
from klampt import *
w = WorldModel()
w.loadFile("../../data/athlete_plane.xml")

True

In [12]:
from klampt import ThreeJSGetScene,ThreeJSGetTransforms
import json
s = ThreeJSGetScene(w)
msg = {'type':'scene','data':json.loads(s)}

my_comm.send(msg)

In [13]:
q = w.robot(0).getConfig()
q[2] += 2
w.robot(0).setConfig(q)
s = ThreeJSGetTransforms(w)
msg = {'type':'transforms','data':json.loads(s)}
my_comm.send(msg)

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, Dict, validate
from IPython.display import display,Javascript

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    msg = Dict({}).tag(sync=True)
    def __init__(self):
        widgets.DOMWidget.__init__(self)
        display(Javascript("""require.undef('hello');

//older version uses jupyter-js-widgets, not @jupyter-widgets/base
define('hello', ["@jupyter-widgets/base"], function(widgets) {
    console.log("Defining hello widget");
    var HelloView = widgets.DOMWidgetView.extend({
        // Render the view.
        render: function() {
            console.log("In render");
            this.value_changed();
            this.model.on('change:msg', this.value_changed, this);
        },
        value_changed: function() {
            console.log(this.model.get('msg'));
            var msg = JSON.stringify(this.model.get('msg'));
            this.el.textContent = 'Got a message of blah length '+msg.length;
            if(msg.length > 0) {
                console.log(msg);
            }
        },
        
    });

    return {
        HelloView: HelloView
    };
});"""))

In [ ]:
w = HelloWidget()
w

In [ ]:
w.msg = {'test1':1}
w.msg = {'test2':2}
w.msg = {'test3':3}
w.msg = {'test4':4}

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, Dict, Int, validate
from IPython.display import display,Javascript

_setup_code = """
function loadScript(url,cb,failcb) {
    jQuery.ajax({
        url: url,
        dataType: 'script',
        success: cb,
        failure: failcb,
        async: true
    });
}
function loadScripts(urls,successcb,failcb) {
    if(urls.length==1) {
        loadScript(urls[0],successcb,function() { alert("Failed to load "+urls[0]); failcb(); });
    }
    else {
        loadScript(urls[0],function() { 
                loadScripts(urls.slice(1,urls.length),successcb,failcb);
            },
            function() {
                alert("Failed to load "+urls[0]);
                failcb();
            });
    }
}
var prefix = "http://klampt.org/";
var klampt_ready = false;
loadScripts([prefix+"three.min.js",
             prefix+"KlamptFrontend.js"],
            function() {
              console.log("Klamp't is ready");
              klampt_ready = true;
            },
            function() {
              console.log("Klamp't failed to load");
            }
);
"""
display(Javascript(_setup_code))

In [ ]:
class KlamptWidget(widgets.DOMWidget):
    _view_name = Unicode('KlamptView').tag(sync=True)
    _view_module = Unicode('klampt-jupyter').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    width = Int(800).tag(sync=True)
    height = Int(600).tag(sync=True)
    msg = Dict({}).tag(sync=True)
    def __init__(self,world=None,*args,**kwargs):
        widgets.DOMWidget.__init__(self,*args,**kwargs)
        self.world = world
        display(Javascript("""require.undef('klampt-jupyter');

//older version uses jupyter-js-widgets, not @jupyter-widgets/base
define('klampt-jupyter', ["@jupyter-widgets/base"], function(widgets) {

    var KlamptView = widgets.DOMWidgetView.extend({
        initialize: function() {
            console.log("Initialize");
            this.delayed_msgs = [];
            this.klampt = null;
        },
        createDiv: function(){
            var width = this.model.get('width');
            var height = this.model.get('height');
            var div = document.createElement("div");
            div.style.width = width+"px";
            div.style.height = height+"px";
            return div;
            },
        // Render the view.
        render: function() {
            console.log("Creating DOM element");
            this.klamptArea = this.createDiv();
            this.el.appendChild(this.klamptArea);
            this.value_changed();
            this.model.on('change:msg', this.value_changed, this);
        },
        value_changed: function() {
            var msg = this.model.get('msg');
            this.klamptArea.textContent = 'Got a message of type '+msg.type;
            if(KLAMPT==null) {
                console.log("klamp't not ready, delaying message of type "+msg.type);
                this.delayed_msgs.push(msg);
            }
            else {
                if(this.klampt == null) {
                    console.log("Creating KLAMPT object");
                    this.klampt = new KLAMPT(this.klamptArea);
                    var _this=this;
                    setTimeout(function() {
                          _this.klampt.resize(_this.model.get('width'),_this.model.get('height'));
                        },
                        500);
                }
                if(this.delayed_msgs.length > 0) {
                    for(var i=0;i<this.delayed_msgs.length;i++) {
                        this.process_msg(this.delayed_msgs[i]);
                    }
                    this.delayed_msgs = [];
                }
                this.process_msg(msg);
            }
        },
        process_msg: function(payload) {
            if(payload['type'] === undefined) {
                return;
            }
            if(payload.type == 'update') {
                console.log("B");
                this.klampt.update_scene(payload.data);
            }
            else if(payload.type == 'scene') {
                console.log("C");
                this.klampt.set_scene(payload.data);
            }
            else if(payload.type == 'rpc') {
                console.log("D");
                this.klampt.rpc(payload.data);
            }
            else {
                this.klamptArea.textContent = "Illegal message type "+payload.type;
            }
        }
        
    });

    return {
        KlamptView: KlamptView
    };
});"""))
        

In [ ]:
w = KlamptWidget()

In [ ]:
w

In [ ]:
w.msg = {'blah':2}

In [ ]:
from IPython.display import display,clear_output,TextDisplayObject
connected = False
value = None
def commfunc(comm,msg):
    global connected,value
    clear_output()
    display(TextDisplayObject("Commfunc "+str(msg)))
    connected = True
    @comm.on_msg
    def _recv(msg):
        global value
        value = msg['content']['data']['blah']
get_ipython().kernel.comm_manager.register_target('test_comm', commfunc)

from IPython.core.display import Javascript
print "HEllo"

Javascript("""
var comm = Jupyter.notebook.kernel.comm_manager.new_comm('test_comm',{});
comm.send({'blah':'2'});
""")

#WEIRD: this doesn't print or update the display inside the javascript callback --
#the kernel is not updating the frontend

In [ ]:
print connected,value